In [1]:
DATA_DIR = '' # input('Enter file path for data files: ')

In [2]:
# Import Libraries
import random
import numpy as np
import pandas as pd

##### **Data Section 3 - TOURNEY**

In [3]:
# Create Games dataframes:
MDResults = pd.read_csv(DATA_DIR + 'MNCAATourneyDetailedResults.csv')
WDResults = pd.read_csv(DATA_DIR + 'WNCAATourneyDetailedResults.csv')

In [4]:
# Create Games dataframes:
MSeeds = pd.read_csv(DATA_DIR + 'MNCAATourneySeeds.csv')
WSeeds = pd.read_csv(DATA_DIR + 'WNCAATourneySeeds.csv')

In [5]:
# Modify 'Seeds' dataframes to add in tournament seeding
seed_dict = {'W01':1, 'W02':2, 'W03':3, 'W04':4, 'W05':5, 'W06':6, 'W07':7, 'W08':8, 
             'W09':9, 'W10':10, 'W11':11, 'W12':12, 'W13':13, 'W14':14, 'W15':15, 'W16':16, 
             'X01':1, 'X02':2, 'X03':3, 'X04':4, 'X05':5, 'X06':6, 'X07':7, 'X08':8, 
             'X09':9, 'X10':10, 'X11':11, 'X12':12, 'X13':13, 'X14':14, 'X15':15, 'X16':16, 
             'Y01':1, 'Y02':2, 'Y03':3, 'Y04':4, 'Y05':5, 'Y06':6, 'Y07':7, 'Y08':8, 
             'Y09':9, 'Y10':10, 'Y11':11, 'Y12':12, 'Y13':13, 'Y14':14, 'Y15':15, 'Y16':16, 
             'Z01':1, 'Z02':2, 'Z03':3, 'Z04':4, 'Z05':5, 'Z06':6, 'Z07':7, 'Z08':8, 
             'Z09':9, 'Z10':10, 'Z11':11, 'Z12':12, 'Z13':13, 'Z14':14, 'Z15':15, 'Z16':16, 
             'W11a':11, 'W11b':11, 'X11a':11, 'X11b':11, 'Y11a':11, 'Y11b':11, 'Z11a':11, 
             'Z11b':11, 'W12a':12, 'W12b':12, 'X12a':12, 'X12b':12, 'Y12a':12, 'Y12b':12, 
             'Z12a':12, 'Z12b':12, 'W13a':13, 'W13b':13, 'X13a':13, 'X13b':13, 'Y13a':13, 
             'Y13b':13, 'Z13a':13, 'Z13b':13, 'W14a':14, 'W14b':14, 'X14a':14, 'X14b':14, 
             'Y14a':14, 'Y14b':14, 'Z14a':14, 'Z14b':14, 'W15a':15, 'W15b':15, 'X15a':15, 
             'X15b':15, 'Y15a':15, 'Y15b':15, 'Z15a':15, 'Z15b':15, 'W16a':16, 'W16b':16, 
             'X16a':16, 'X16b':16, 'Y16a':16, 'Y16b':16, 'Z16a':16, 'Z16b':16}

mens_probs = {1: 0.864, 2: 0.779, 3: 0.685, 4: 0.635, 5: 0.539, 6: 0.535, 
              7: 0.444, 8: 0.443, 9: 0.355, 10: 0.352, 11: 0.207, 12: 0.188, 
              13: 0.145, 14: 0.086, 15: 0.033, 16: 0.011}

womens_probs = {1: 0.870, 2: 0.798, 3: 0.705, 4: 0.645, 5: 0.541, 6: 0.540, 
                7: 0.448, 8: 0.449, 9: 0.365, 10: 0.346, 11: 0.191, 12: 0.214, 
                13: 0.151, 14: 0.074, 15: 0.032, 16: 0.007}

# Create a new columns 'SeedNum' and 'SeedProb' based on mapping values to dictionary
MSeeds['SeedNum'] = MSeeds['Seed'].map(seed_dict)
MSeeds['SeedProb'] = MSeeds['SeedNum'].map(mens_probs)
WSeeds['SeedNum'] = WSeeds['Seed'].map(seed_dict)
WSeeds['SeedProb'] = WSeeds['SeedNum'].map(womens_probs)

# Drop unnecessary column
MSeeds = MSeeds.drop(columns=['Seed'])
WSeeds = WSeeds.drop(columns=['Seed'])
MSeeds.columns

Index(['Season', 'TeamID', 'SeedNum', 'SeedProb'], dtype='object')

In [6]:
# Create Conference Abbreviations dataframes:
MTeamConferences = pd.read_csv(DATA_DIR + 'MTeamConferences.csv')
WTeamConferences = pd.read_csv(DATA_DIR + 'WTeamConferences.csv')

In [7]:
# Create Favorites dataframes (for subsequent merge and build):
MFavorites = MDResults.loc[MDResults['Season'] >= 2010, ['Season', 'DayNum', 'WTeamID', 'LTeamID', 'WLoc']].copy()
WFavorites = WDResults.loc[WDResults['Season'] >= 2010, ['Season', 'DayNum', 'WTeamID', 'LTeamID', 'WLoc']].copy()

In [8]:
# Create winning percentage statistics from (sport-reference.com) dataframes:
MWinningPCT = pd.read_csv(DATA_DIR + 'MWinningPCT.csv')

In [9]:
# Show sample output
print('Mens:', MFavorites.shape, 'Womens:', WFavorites.shape)
print('Mens:', MFavorites.columns)

Mens: (800, 5) Womens: (760, 5)
Mens: Index(['Season', 'DayNum', 'WTeamID', 'LTeamID', 'WLoc'], dtype='object')


In [10]:
# Merge 'Favorites' with 'Seeds' dataframes on 'Season' and 'WTeamID' columns
MFavorites_df = pd.merge(MFavorites, MSeeds, how='left', left_on=['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])
WFavorites_df = pd.merge(WFavorites, WSeeds, how='left', left_on=['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])

# Rename, reorder, and drop unwanted columns from merged dataframe
rename = {'SeedNum':'WSeedNum', 'SeedProb':'WSeedProb'}

MFavorites_df = MFavorites_df.drop(columns=['TeamID'])
WFavorites_df = WFavorites_df.drop(columns=['TeamID'])
MFavorites_df = MFavorites_df.rename(columns=rename)
WFavorites_df = WFavorites_df.rename(columns=rename)

# Perform second merge this time on 'Season' and 'LTeamID' columns
MFavorites2_df = pd.merge(MFavorites_df, MSeeds, how='left', left_on=['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])
WFavorites2_df = pd.merge(WFavorites_df, WSeeds, how='left', left_on=['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])

# Rename and drop unwanted and columns from merged dataframe
rename = {'SeedNum':'LSeedNum', 'SeedProb':'LSeedProb'}

MFavorites2_df = MFavorites2_df.drop(columns=['TeamID'])
WFavorites2_df = WFavorites2_df.drop(columns=['TeamID'])
MFavorites2_df = MFavorites2_df.rename(columns=rename)
WFavorites2_df = WFavorites2_df.rename(columns=rename)

# Show sample output
print('Mens:', MFavorites2_df.shape, 'Womens:', WFavorites2_df.shape)
print('Mens:', MFavorites2_df.columns)
MFavorites2_df.tail()

Mens: (800, 9) Womens: (760, 9)
Mens: Index(['Season', 'DayNum', 'WTeamID', 'LTeamID', 'WLoc', 'WSeedNum',
       'WSeedProb', 'LSeedNum', 'LSeedProb'],
      dtype='object')


,Season,DayNum,WTeamID,LTeamID,WLoc,WSeedNum,WSeedProb,LSeedNum,LSeedProb
795,2022,146,1242,1274,N,1,0.864,10,0.352
796,2022,146,1314,1389,N,8,0.443,15,0.033
797,2022,152,1242,1437,N,1,0.864,2,0.779
798,2022,152,1314,1181,N,8,0.443,2,0.779
799,2022,154,1242,1314,N,1,0.864,8,0.443


In [11]:
# Create new columns (Mens) based on the value of the 'WLoc' column
MFavorites2_df['WLocH'] = 1
MFavorites2_df['WLocA'] = 1
MFavorites2_df['WLocG'] = 1
MFavorites2_df['LLocH'] = 1
MFavorites2_df['LLocA'] = 1
MFavorites2_df['LLocG'] = 1

# Create new columns (Womens) based on the value of the 'WLoc' column
WFavorites2_df['WLocH'] = 1
WFavorites2_df['WLocA'] = 1
WFavorites2_df['WLocG'] = 1
WFavorites2_df['LLocH'] = 1
WFavorites2_df['LLocA'] = 1
WFavorites2_df['LLocG'] = 1

# Show sample output
print('Mens:', MFavorites2_df.shape, 'Womens:', WFavorites2_df.shape)
print('Mens:', MFavorites2_df.columns)
MFavorites2_df.tail()

Mens: (800, 15) Womens: (760, 15)
Mens: Index(['Season', 'DayNum', 'WTeamID', 'LTeamID', 'WLoc', 'WSeedNum',
       'WSeedProb', 'LSeedNum', 'LSeedProb', 'WLocH', 'WLocA', 'WLocG',
       'LLocH', 'LLocA', 'LLocG'],
      dtype='object')


,Season,DayNum,WTeamID,LTeamID,WLoc,WSeedNum,WSeedProb,LSeedNum,LSeedProb,WLocH,WLocA,WLocG,LLocH,LLocA,LLocG
795,2022,146,1242,1274,N,1,0.864,10,0.352,1,1,1,1,1,1
796,2022,146,1314,1389,N,8,0.443,15,0.033,1,1,1,1,1,1
797,2022,152,1242,1437,N,1,0.864,2,0.779,1,1,1,1,1,1
798,2022,152,1314,1181,N,8,0.443,2,0.779,1,1,1,1,1,1
799,2022,154,1242,1314,N,1,0.864,8,0.443,1,1,1,1,1,1


In [12]:
# Merge 'Favorites' with 'Team Conferences' dataframes on 'Season' and 'WTeamID' columns
MFavorites3_df = pd.merge(MFavorites2_df, MTeamConferences, how='left', left_on=['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])
WFavorites3_df = pd.merge(WFavorites2_df, WTeamConferences, how='left', left_on=['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])

# Rename and drop unwanted columns from merged dataframe
rename = {'ConfAbbrev':'WConf'}

MFavorites3_df = MFavorites3_df.drop(columns=['WLoc', 'TeamID'])
WFavorites3_df = WFavorites3_df.drop(columns=['WLoc', 'TeamID'])
MFavorites3_df = MFavorites3_df.rename(columns=rename)
WFavorites3_df = WFavorites3_df.rename(columns=rename)

# Perform second merge this time on 'Season' and 'LTeamID' columns
MFavorites4_df = pd.merge(MFavorites3_df, MTeamConferences, how='left', left_on=['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])
WFavorites4_df = pd.merge(WFavorites3_df, WTeamConferences, how='left', left_on=['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])

# Rename and drop unwanted and columns from merged dataframe
rename = {'ConfAbbrev':'LConf'}

MFavorites4_df = MFavorites4_df.drop(columns=['TeamID'])
WFavorites4_df = WFavorites4_df.drop(columns=['TeamID'])
MFavorites4_df = MFavorites4_df.rename(columns=rename)
WFavorites4_df = WFavorites4_df.rename(columns=rename)

# Show sample output
print('Mens:', MFavorites4_df.shape, 'Womens:', WFavorites4_df.shape)
MFavorites4_df.head()

Mens: (800, 16) Womens: (760, 16)


,Season,DayNum,WTeamID,LTeamID,WSeedNum,WSeedProb,LSeedNum,LSeedProb,WLocH,WLocA,WLocG,LLocH,LLocA,LLocG,WConf,LConf
0,2010,134,1115,1457,16,0.011,16,0.011,1,1,1,1,1,1,swac,big_south
1,2010,136,1124,1358,3,0.685,14,0.086,1,1,1,1,1,1,big_twelve,southland
2,2010,136,1139,1431,5,0.539,12,0.188,1,1,1,1,1,1,horizon,cusa
3,2010,136,1140,1196,7,0.444,10,0.352,1,1,1,1,1,1,mwc,sec
4,2010,136,1242,1250,1,0.864,16,0.011,1,1,1,1,1,1,big_twelve,patriot


In [13]:
# Merge 'Favorites' with 'Team Conferences' dataframes on 'Season' and 'WTeamID' columns
MFavorites5_df = pd.merge(MFavorites4_df, MWinningPCT, how='left', left_on=['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])
#WFavorites5_df = pd.merge(WFavorites4_df, WWinningPCT, how='left', left_on=['Season', 'WTeamID'], right_on = ['Season', 'TeamID'])

# Rename, reorder, and drop unwanted columns from merged dataframe
rename = {'WinPCT':'WWinPCT', 'ConfPCT':'WConfPCT', 'HomePCT':'WHomePCT', 'AwayPCT':'WAwayPCT'}

MFavorites5_df = MFavorites5_df.drop(columns=['TeamID'])
#WFavorites5_df = WFavorites5_df.drop(columns=['TeamID'])
MFavorites5_df = MFavorites5_df.rename(columns=rename)
#WFavorites5_df = WFavorites5_df.rename(columns=rename)

# Perform second merge this time on 'Season' and 'LTeamID' columns
MFavorites6_df = pd.merge(MFavorites5_df, MWinningPCT, how='left', left_on=['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])
#WFavorites6_df = pd.merge(WFavorites5_df, WWinningPCT, how='left', left_on=['Season', 'LTeamID'], right_on = ['Season', 'TeamID'])

# Rename, reorder, and drop unwanted columns from merged dataframe
rename = {'WinPCT':'LWinPCT', 'ConfPCT':'LConfPCT', 'HomePCT':'LHomePCT', 'AwayPCT':'LAwayPCT'}

MFavorites6_df = MFavorites6_df.drop(columns=['TeamID'])
#WFavorites6_df = WFavorites6_df.drop(columns=['TeamID'])
MFavorites6_df = MFavorites6_df.rename(columns=rename)
#WFavorites6_df = WFavorites6_df.rename(columns=rename)

# Show sample output
print('Mens:', MFavorites6_df.shape)#, 'Womens:', WFavorites6_df.shape)
MFavorites6_df.head()

Mens: (800, 24)


,Season,DayNum,WTeamID,LTeamID,WSeedNum,WSeedProb,LSeedNum,LSeedProb,WLocH,WLocA,...,WConf,LConf,WWinPCT,WConfPCT,WHomePCT,WAwayPCT,LWinPCT,LConfPCT,LHomePCT,LAwayPCT
0,2010,134,1115,1457,16,0.011,16,0.011,1,1,...,swac,big_south,0.529412,0.777778,0.888889,0.263158,0.575758,0.666667,0.857143,0.352941
1,2010,136,1124,1358,3,0.685,14,0.086,1,1,...,big_twelve,southland,0.777778,0.687500,0.937500,0.600000,0.757576,0.875000,0.928571,0.538462
2,2010,136,1139,1431,5,0.539,12,0.188,1,1,...,horizon,cusa,0.868421,1.000000,1.000000,0.923077,0.787879,0.937500,0.882353,0.800000
3,2010,136,1140,1196,7,0.444,10,0.352,1,1,...,mwc,sec,0.833333,0.812500,0.937500,0.714286,0.617647,0.562500,0.764706,0.500000
4,2010,136,1242,1250,1,0.864,16,0.011,1,1,...,big_twelve,patriot,0.916667,0.937500,1.000000,0.818182,0.666667,0.714286,0.882353,0.466667


In [14]:
# Keep MFavorites6_df['WWinPCT'] and MFavorites6_df['LWinPCT']
# Keep WFavorites6_df['WWinPCT'] and MFavorites6_df['LWinPCT']

# Change the value of 'ConfPCT' based on the equality of the 'WConf' and 'LConf' columns
MFavorites6_df['WConfPCT'] = np.where(MFavorites6_df['WConf'] == MFavorites6_df['LConf'], MFavorites6_df['WConfPCT'], 1 - MFavorites6_df['WConfPCT'])
#WFavorites6_df['WConfPCT'] = np.where(WFavorites6_df['WConf'] == WFavorites6_df['LConf'], WFavorites6_df['WConfPCT'], 1 - WFavorites6_df['WConfPCT'])
MFavorites6_df['LConfPCT'] = np.where(MFavorites6_df['WConf'] == MFavorites6_df['LConf'], 1 - MFavorites6_df['LConfPCT'], MFavorites6_df['LConfPCT'])
#WFavorites6_df['LConfPCT'] = np.where(WFavorites6_df['WConf'] == WFavorites6_df['LConf'], 1 - WFavorites6_df['LConfPCT'], wFavorites6_df['LConfPCT'])

# Create a new column based on the equality of the 'WConf' and 'LConf' columns
MFavorites6_df['WHomePCT'] = np.where(MFavorites6_df['WSeedNum'] < MFavorites6_df['LSeedNum'], MFavorites6_df['WSeedProb'], 1 - MFavorites6_df['WSeedProb'])
#WFavorites6_df['WHomePCT'] = np.where(WFavorites6_df['WSeedNum'] < WFavorites6_df['LSeedNum'], WFavorites6_df['WSeedProb'], 1 - WFavorites6_df['WSeedProb'])
MFavorites6_df['LHomePCT'] = np.where(MFavorites6_df['WSeedNum'] >= MFavorites6_df['LSeedNum'], MFavorites6_df['LSeedProb'], 1 - MFavorites6_df['LSeedProb'])
#WFavorites6_df['LHomePCT'] = np.where(WFavorites6_df['WSeedNum'] >= WFavorites6_df['LSeedNum'], WFavorites6_df['LSeedProb'], 1 - wFavorites6_df['LSeedProb'])

# Create a new column based on the equality of the 'WConf' and 'LConf' columns
MFavorites6_df['WAwayPCT'] = np.where(MFavorites6_df['WSeedNum'] >= MFavorites6_df['LSeedNum'], 1 - MFavorites6_df['WConfPCT'], MFavorites6_df['WConfPCT'])
#WFavorites6_df['WAwayPCT'] = np.where(WFavorites6_df['WSeedNum'] >= WFavorites6_df['LSeedNum'], 1 - WFavorites6_df['WConfPCT'], WFavorites6_df['WConfPCT'])
MFavorites6_df['LAwayPCT'] = np.where(MFavorites6_df['WSeedNum'] < MFavorites6_df['LSeedNum'], 1 - MFavorites6_df['LConfPCT'], MFavorites6_df['LConfPCT'])
#WFavorites6_df['LAwayPCT'] = np.where(WFavorites6_df['WSeedNum'] < WFavorites6_df['LSeedNum'], 1 - WFavorites6_df['LConfPCT'], wFavorites6_df['LConfPCT'])

# Create four new columns (Mens) to calculate the winning percentages and determine if winning team was the favorite
MFavorites6_df['WTPCT'] = (MFavorites6_df['WWinPCT'] + MFavorites6_df['WConfPCT'] + MFavorites6_df['WHomePCT'] + MFavorites6_df['WAwayPCT']) / 4
#WFavorites6_df['WTPCT'] = (WFavorites6_df['WWinPCT'] + WFavorites6_df['WConfPCT'] + WFavorites6_df['WHomePCT'] + WFavorites6_df['WAwayPCT']) / 4
MFavorites6_df['LTPCT'] = (MFavorites6_df['LWinPCT'] + MFavorites6_df['LConfPCT'] + MFavorites6_df['LHomePCT'] + MFavorites6_df['LAwayPCT']) / 4
#WFavorites6_df['LTPCT'] = (WFavorites6_df['LWinPCT'] + WFavorites6_df['LConfPCT'] + WFavorites6_df['LHomePCT'] + WFavorites6_df['LAwayPCT']) / 4

MFavorites6_df['WFAV'] = (MFavorites6_df['WTPCT'] - MFavorites6_df['LTPCT']).ge(0).astype(int)
#WFavorites6_df['WFAV'] = (WFavorites6_df['WTPCT'] - WFavorites6_df['LTPCT']).ge(0).astype(int)
MFavorites6_df['LFAV'] = (MFavorites6_df['WTPCT'] - MFavorites6_df['LTPCT']).lt(0).astype(int)
#WFavorites6_df['LFAV'] = (WFavorites6_df['WTPCT'] - WFavorites6_df['LTPCT']).lt(0).astype(int)

# Show sample output
print('Mens:', MFavorites6_df.shape)#, 'Womens:', WFavorites6_df.shape)
MFavorites6_df.head()

Mens: (800, 28)


,Season,DayNum,WTeamID,LTeamID,WSeedNum,WSeedProb,LSeedNum,LSeedProb,WLocH,WLocA,...,WHomePCT,WAwayPCT,LWinPCT,LConfPCT,LHomePCT,LAwayPCT,WTPCT,LTPCT,WFAV,LFAV
0,2010,134,1115,1457,16,0.011,16,0.011,1,1,...,0.989,0.777778,0.575758,0.666667,0.011,0.666667,0.629603,0.480023,1,0
1,2010,136,1124,1358,3,0.685,14,0.086,1,1,...,0.685,0.312500,0.757576,0.875000,0.914,0.125000,0.521944,0.667894,0,1
2,2010,136,1139,1431,5,0.539,12,0.188,1,1,...,0.539,0.000000,0.787879,0.937500,0.812,0.062500,0.351855,0.649970,0,1
3,2010,136,1140,1196,7,0.444,10,0.352,1,1,...,0.444,0.187500,0.617647,0.562500,0.648,0.437500,0.413083,0.566412,0,1
4,2010,136,1242,1250,1,0.864,16,0.011,1,1,...,0.864,0.062500,0.666667,0.714286,0.989,0.285714,0.476417,0.663917,0,1


In [15]:
# Bring back in original 'Games' dataframe (for subsequent merge and build):
MFavorites7_df = MDResults.loc[MDResults['Season'] >= 2010].copy()
WFavorites7_df = WDResults.loc[WDResults['Season'] >= 2010].copy()

# Merge 'Favorites' with original 'Games' dataframes on 'Season' and 'WTeamID' columns
MFavorites8_df = pd.merge(MFavorites6_df, MFavorites7_df, how='left', left_on=['Season', 'DayNum', 'WTeamID', 'LTeamID'], right_on = ['Season', 'DayNum', 'WTeamID', 'LTeamID'])
#WFavorites8_df = pd.merge(WFavorites6_df, WFavorites7_df, how='left', left_on=['Season', 'DayNum', 'WTeamID', 'LTeamID'], right_on = ['Season', 'DayNum', 'WTeamID', 'LTeamID'])

# Show sample output
print('Mens:', MFavorites8_df.shape)#, 'Womens:', WFavorites8_df.shape)
MFavorites8_df.head()

Mens: (800, 58)


,Season,DayNum,WTeamID,LTeamID,WSeedNum,WSeedProb,LSeedNum,LSeedProb,WLocH,WLocA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,134,1115,1457,16,0.011,16,0.011,1,1,...,21,10,18,14,20,5,7,3,5,22
1,2010,136,1124,1358,3,0.685,14,0.086,1,1,...,31,7,13,8,29,20,11,7,1,20
2,2010,136,1139,1431,5,0.539,12,0.188,1,1,...,18,7,9,6,24,15,14,8,1,24
3,2010,136,1140,1196,7,0.444,10,0.352,1,1,...,32,14,19,20,25,15,21,7,4,24
4,2010,136,1242,1250,1,0.864,16,0.011,1,1,...,19,17,21,13,20,11,13,6,4,16


In [16]:
# Split merged 'Favorites' (version 8) dataframes (for subsequent build and matchups):

MFavorites9_df = MFavorites8_df.loc[:, ['Season', 'DayNum', 'WTeamID', 'LTeamID', 'WTPCT', 'LTPCT', 'WFAV', 'LFAV', 'WScore', 'LScore', 
                                        'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 
                                        'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 
                                        'LTO', 'LStl', 'LBlk', 'LPF']].copy()

split_win = MFavorites9_df.loc[:, ['Season', 'DayNum', 'WTeamID', 'WTPCT', 'WFAV', 'WScore', 'LScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 
                                   'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']].copy()


split_loss = MFavorites9_df.loc[:, ['Season', 'DayNum', 'LTeamID', 'LTPCT', 'LFAV', 'WScore', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 
                                    'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']].copy()

# Rename columns to prepare for concatenate dataframes
rename_win = {'WTeamID':'TeamID', 'WTPCT':'TPCT', 'WFAV':'WIN', 'WFGM':'FGM', 'WFGA':'FGA', 'WFGM3':'FGM3', 'WFGA3':'FGA3', 
              'WFTM':'FTM', 'WFTA':'FTA', 'WOR':'OR', 'WDR':'DR', 'WAst':'AST', 'WTO':'TOV', 'WStl':'STL', 'WBlk':'BLK', 'WPF':'PF'}

rename_loss = {'LTeamID':'TeamID', 'LTPCT':'TPCT', 'LFAV':'WIN', 'LFGM':'FGM', 'LFGA':'FGA', 'LFGM3':'FGM3', 'LFGA3':'FGA3', 
              'LFTM':'FTM', 'LFTA':'FTA', 'LOR':'OR', 'LDR':'DR', 'LAst':'AST', 'LTO':'TOV', 'LStl':'STL', 'LBlk':'BLK', 'LPF':'PF'}

split_win = split_win.rename(columns=rename_win)
split_loss = split_loss.rename(columns=rename_loss)

# Concatenate the two dataframes
MFavorites10_df = pd.concat([split_win, split_loss])
MFavorites10_df = MFavorites10_df.sort_values(by=['Season', 'DayNum', 'TeamID'])

In [17]:
# Show sample output
print('Mens:', MFavorites10_df.shape)#, 'Womens:', WFavorites10_df.shape)

# select rows where 'Season' is 2021, 'WTeamID' is 456, and 'LTeamID' is 123
MFavorites10_df.loc[(MFavorites10_df['Season'] == 2013) & (MFavorites10_df['DayNum'] == 136) & ((MFavorites10_df['TeamID'] == 1387) | (MFavorites10_df['TeamID'] == 1308)), :]

Mens: (1600, 20)


,Season,DayNum,TeamID,TPCT,WIN,WScore,LScore,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,AST,TOV,STL,BLK,PF
214,2013,136,1308,0.635179,1,64,44,17,61,2,16,8,13,21,21,10,16,9,7,12
214,2013,136,1387,0.452500,0,64,44,25,54,4,11,10,14,10,23,17,10,10,3,15


In [18]:
print('Missing Values:', MFavorites10_df.isna().sum().sum())

Missing Values: 0


In [19]:
# Drop rows where total winning percentage is missing, but exclude the 2023 Season
#mens_dropped = MFavorites10_df.loc[MFavorites8_df.TPCT.notnull(), :].copy()
#print('Missing Values:', mens_dropped.isna().sum().sum())

In [20]:
# Combine related columns
MFavorites10_df['FGPCT'] = MFavorites10_df['FGM'] / MFavorites10_df['FGA']
MFavorites10_df['FG3PCT'] = MFavorites10_df['FGM3'] / MFavorites10_df['FGA3']
MFavorites10_df['FTPCT'] = MFavorites10_df['FTM'] / MFavorites10_df['FTA']
MFavorites10_df['Luck'] = [random.uniform(0, 1) for _ in range(len(MFavorites10_df))]

#WFavorites10_df['FGPCT'] = WFavorites10_df['FGM'] / WFavorites10_df['FGA']
#WFavorites10_df['FG3PCT'] = WFavorites10_df['FGM3'] / WFavorites10_df['FGA3']
#WFavorites10_df['FTPCT'] = WFavorites10_df['FTM'] / WFavorites10_df['FTA']
#WFavorites10_df['Luck'] = [random.uniform(0, 1) for _ in range(len(WFavorites10_df))]

# Reorder and drop unwanted columns from merged dataframe
order = [0, 1, 2, 3, 23, 5, 6, 20, 7, 8, 21, 9, 10, 22, 11, 12, 13, 14, 15, 16, 17, 18, 19, 4]

MFavorites10_df = MFavorites10_df.iloc[:,order].drop(columns=['FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA'])
#WFavorites10_df = WFavorites10_df.iloc[:,order].drop(columns=['FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA'])

# Show sample output
print('Mens:', MFavorites10_df.shape)#, 'Womens:', WFavorites10_df.shape)
MFavorites10_df.head(3)

Mens: (1600, 18)


,Season,DayNum,TeamID,TPCT,Luck,WScore,LScore,FGPCT,FG3PCT,FTPCT,OR,DR,AST,TOV,STL,BLK,PF,WIN
0,2010,134,1115,0.629603,0.897974,61,44,0.386364,0.375000,0.750000,10,29,12,10,3,7,16,1
0,2010,134,1457,0.480023,0.778367,61,44,0.290909,0.095238,0.555556,14,20,5,7,3,5,22,0
1,2010,136,1124,0.521944,0.102560,68,59,0.440678,0.250000,0.647059,8,39,18,14,8,2,18,0


In [21]:
# Normalize selected columns using min-max range
def normalize_minmax(column):
    """
    Normalize a column to a min-max range.
    """
    min_val = min(column)
    max_val = max(column)
    normalized_col = [(x - min_val) / (max_val - min_val) for x in column]
    return normalized_col

# Select the columns you want to normalize
selected_cols = ['OR', 'DR', 'AST', 'TOV', 'STL', 'BLK', 'PF']

# Normalize the selected columns in your dataframe
for col in selected_cols:
    MFavorites10_df[col] = normalize_minmax(MFavorites10_df[col])

# Show sample output
print('Mens:', MFavorites10_df.shape)#, 'Womens:', WFavorites10_df.shape)
MFavorites10_df.head(3)

Mens: (1600, 18)


,Season,DayNum,TeamID,TPCT,Luck,WScore,LScore,FGPCT,FG3PCT,FTPCT,OR,DR,AST,TOV,STL,BLK,PF,WIN
0,2010,134,1115,0.629603,0.897974,61,44,0.386364,0.375000,0.750000,0.344828,0.600000,0.370370,0.307692,0.15,0.466667,0.346154,1
0,2010,134,1457,0.480023,0.778367,61,44,0.290909,0.095238,0.555556,0.482759,0.342857,0.111111,0.192308,0.15,0.333333,0.576923,0
1,2010,136,1124,0.521944,0.102560,68,59,0.440678,0.250000,0.647059,0.275862,0.885714,0.592593,0.461538,0.40,0.133333,0.423077,0


In [22]:
# Standardize selected columns using standard scaler
def standardize_scaler(column):
    """
    Standardize a column using standard scaler.
    """
    mean_val = sum(column) / len(column)
    std_val = (sum([(x - mean_val)**2 for x in column]) / (len(column)-1))**0.5
    standardized_col = [(x - mean_val) / std_val for x in column]
    return standardized_col

# Select the columns you want to standardize
selected_cols = ['WScore', 'LScore']

# Standardize the selected columns in your dataframe
for col in selected_cols:
    MFavorites10_df[col] = standardize_scaler(MFavorites10_df[col])
    
# Show sample output
print('Mens:', MFavorites10_df.shape)#, 'Womens:', WFavorites10_df.shape)
MFavorites10_df.head(3)

Mens: (1600, 18)


,Season,DayNum,TeamID,TPCT,Luck,WScore,LScore,FGPCT,FG3PCT,FTPCT,OR,DR,AST,TOV,STL,BLK,PF,WIN
0,2010,134,1115,0.629603,0.897974,-1.323457,-1.910590,0.386364,0.375000,0.750000,0.344828,0.600000,0.370370,0.307692,0.15,0.466667,0.346154,1
0,2010,134,1457,0.480023,0.778367,-1.323457,-1.910590,0.290909,0.095238,0.555556,0.482759,0.342857,0.111111,0.192308,0.15,0.333333,0.576923,0
1,2010,136,1124,0.521944,0.102560,-0.654743,-0.429512,0.440678,0.250000,0.647059,0.275862,0.885714,0.592593,0.461538,0.40,0.133333,0.423077,0


In [23]:
tourney_mens = MFavorites10_df.to_csv('tourney-mens.csv',index=False)
#tourney_womens = WFavorites10_df.to_csv('tourney-womens.csv',index=False)